## Week 1

### Introduction / Business Problem

For my project, I'd like to consider opening a bar in Atlanta. There are a number of factors to consider when opening up a bar, with one of the most consequential decisions being location. In this hypothetical, the bar would be aimed at young professionals who are looking for somewhere to drink when they get off work. I will be looking for locations that have a high concentration of young and middle-aged professionals with a mid-to-high degree of disposable income. 

### Data

To solve this problem, we will need location data on commercial areas in atlanta. In Atlanta, as well as in other major cities, banks and law firms usually employ people that have a high degree of disposable income. In addition to these locations, colleges usually have a number of professors and other staff that make a significant amount of money. These offices are also usually in locations that have a high degree of commercial traffic. For this hypothetical, we will focus on constricting a foursquare map that reflects the locations of banks, law offices, and colleges in Atlanta. To do so, we pull data on atlanta neighborhoods from https://opendata.atlantaregional.com/datasets/d6298dee8938464294d3f49d473bcf15_196?geometry=-85.289%2C33.567%2C-83.551%2C33.967 to construct the map of Atlanta. We then use the Foursquare service to pull bank, law office, and college locations. Using k-means clustering, we find 3 ideal locations for a bar. 

### Methodology

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [2]:
atlanta_data = json.loads(requests.get('https://opendata.arcgis.com/datasets/d6298dee8938464294d3f49d473bcf15_196.geojson').text)

In [3]:
atlanta_data

{'type': 'FeatureCollection',
 'name': 'City_of_Atlanta_Neighborhood_Statistical_Areas',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1,
    'NPU': 'NPU C',
    'STATISTICA': 'C04',
    'POP2010': 2672,
    'NEIGHBORHO': 'Arden/Habersham, Argonne Forest, Peachtree Battle Alliance, Wyngate',
    'URL': 'http://documents.atlantaregional.com/AtlantaProfiles/C04.pdf',
    'A': 'Arden/Habersham, Argonne Forest, Peachtree Battle Alliance, Wyngate',
    'pop': 2672,
    'white': 96.3,
    'black': 0.9,
    'asian': 1.0,
    'other': 0.5,
    'hispanic': 1.3,
    'GlobalID': '{6018B100-48DF-4B88-85B7-C4095D15897D}',
    'last_edited_date': '1970/01/01 00:00:00+00'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-84.40728321312353, 33.8445284540347],
      [-84.40735621368728, 33.84217845397712],
      [-84.40738221308668, 33.842020453850736],
      [-84.40745021353541, 33.84185645

In [4]:
atlanta_data = atlanta_data['features']

In [5]:
atlanta_data[0]

{'type': 'Feature',
 'properties': {'OBJECTID': 1,
  'NPU': 'NPU C',
  'STATISTICA': 'C04',
  'POP2010': 2672,
  'NEIGHBORHO': 'Arden/Habersham, Argonne Forest, Peachtree Battle Alliance, Wyngate',
  'URL': 'http://documents.atlantaregional.com/AtlantaProfiles/C04.pdf',
  'A': 'Arden/Habersham, Argonne Forest, Peachtree Battle Alliance, Wyngate',
  'pop': 2672,
  'white': 96.3,
  'black': 0.9,
  'asian': 1.0,
  'other': 0.5,
  'hispanic': 1.3,
  'GlobalID': '{6018B100-48DF-4B88-85B7-C4095D15897D}',
  'last_edited_date': '1970/01/01 00:00:00+00'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-84.40728321312353, 33.8445284540347],
    [-84.40735621368728, 33.84217845397712],
    [-84.40738221308668, 33.842020453850736],
    [-84.40745021353541, 33.84185645344486],
    [-84.40749921310712, 33.84177945347297],
    [-84.4075682136065, 33.84169545317609],
    [-84.40767321307789, 33.8415994532212],
    [-84.40804721333068, 33.84131445349337],
    [-84.40816021317214, 33.8411974534646

In [6]:
atlanta_data[1]

{'type': 'Feature',
 'properties': {'OBJECTID': 2,
  'NPU': 'NPU B',
  'STATISTICA': 'B10',
  'POP2010': 3736,
  'NEIGHBORHO': 'Peachtree Heights East, Peachtree Hills',
  'URL': 'http://documents.atlantaregional.com/AtlantaProfiles/B10.pdf',
  'A': 'Peachtree Heights East, Peachtree Hills',
  'pop': 3736,
  'white': 86.6,
  'black': 5.9,
  'asian': 2.0,
  'other': 1.8,
  'hispanic': 3.7,
  'GlobalID': '{FA9A3ABB-FA59-43CD-92A8-B3BFCA21CB1D}',
  'last_edited_date': '1970/01/01 00:00:00+00'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-84.37318920267784, 33.821016450600794],
    [-84.37394820283187, 33.8211444502074],
    [-84.37432620239075, 33.82122845048765],
    [-84.37498420325488, 33.82137545007897],
    [-84.37514020330174, 33.82142045035937],
    [-84.37529220316353, 33.82147545020383],
    [-84.3756712027733, 33.82164444992543],
    [-84.37577920329078, 33.821690450253925],
    [-84.376628203141, 33.822125450590356],
    [-84.37758120332349, 33.822614450726704],
    [

In [7]:
column_names = ['Neighborhood','Census Data','Population', 'Latitude','Longitude']
Atlanta = pd.DataFrame(columns = column_names)

In [8]:
Atlanta

,Neighborhood,Census Data,Population,Latitude,Longitude


In [9]:
for data in atlanta_data:
    neighborhood = data['properties']['NEIGHBORHO']
    census_data = data['properties']['URL']
    population = data['properties']['POP2010']
        
    atlanta_latlon = data['geometry']['coordinates']
    atlanta_lat = atlanta_latlon[0][0][0]
    atlanta_lon = atlanta_latlon[0][0][1]
    
    Atlanta = Atlanta.append({'Neighborhood':neighborhood,
                                          'Census Data': census_data,
                                          'Population':population,
                                          'Latitude': atlanta_lat,
                                          'Longitude': atlanta_lon
                                           },
                                            ignore_index=True)

In [10]:
Atlanta.head(10)

,Neighborhood,Census Data,Population,Latitude,Longitude
0,"Arden/Habersham, Argonne Forest, Peachtree Bat...",http://documents.atlantaregional.com/AtlantaPr...,2672,-84.4073,33.8445
1,"Peachtree Heights East, Peachtree Hills",http://documents.atlantaregional.com/AtlantaPr...,3736,-84.3732,33.821
2,Peachtree Heights West,http://documents.atlantaregional.com/AtlantaPr...,4874,-84.3846,33.8417
3,"Buckhead Forest, South Tuxedo Park",http://documents.atlantaregional.com/AtlantaPr...,3372,-84.3806,33.8396
4,"Chastain Park, Tuxedo Park",http://documents.atlantaregional.com/AtlantaPr...,3423,-84.4073,33.8494
5,East Chastain Park,http://documents.atlantaregional.com/AtlantaPr...,2092,-84.3824,33.8781
6,Garden Hills,http://documents.atlantaregional.com/AtlantaPr...,3584,-84.3832,33.8336
7,Lindbergh/Morosgo,http://documents.atlantaregional.com/AtlantaPr...,4604,-84.3734,33.8158
8,"Buckhead Village, Peachtree Park",http://documents.atlantaregional.com/AtlantaPr...,2920,-84.3803,33.8371
9,Vine City,http://documents.atlantaregional.com/AtlantaPr...,2818,-84.4075,33.7547


In [11]:
address = 'Atlanta, GA'

geolocator = Nominatim(user_agent='atl_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Atlanta is {}, {}.'.format(latitude, longitude))

The geographical coordinate of Atlanta is 33.7490987, -84.3901849.


In [12]:
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=12)


In [13]:
map_atlanta

In [14]:
Atlanta

,Neighborhood,Census Data,Population,Latitude,Longitude
0,"Arden/Habersham, Argonne Forest, Peachtree Bat...",http://documents.atlantaregional.com/AtlantaPr...,2672,-84.4073,33.8445
1,"Peachtree Heights East, Peachtree Hills",http://documents.atlantaregional.com/AtlantaPr...,3736,-84.3732,33.821
2,Peachtree Heights West,http://documents.atlantaregional.com/AtlantaPr...,4874,-84.3846,33.8417
3,"Buckhead Forest, South Tuxedo Park",http://documents.atlantaregional.com/AtlantaPr...,3372,-84.3806,33.8396
4,"Chastain Park, Tuxedo Park",http://documents.atlantaregional.com/AtlantaPr...,3423,-84.4073,33.8494
5,East Chastain Park,http://documents.atlantaregional.com/AtlantaPr...,2092,-84.3824,33.8781
6,Garden Hills,http://documents.atlantaregional.com/AtlantaPr...,3584,-84.3832,33.8336
7,Lindbergh/Morosgo,http://documents.atlantaregional.com/AtlantaPr...,4604,-84.3734,33.8158
8,"Buckhead Village, Peachtree Park",http://documents.atlantaregional.com/AtlantaPr...,2920,-84.3803,33.8371
9,Vine City,http://documents.atlantaregional.com/AtlantaPr...,2818,-84.4075,33.7547


In [15]:
CLIENT_ID = 'T0LRXY51HVVKPRWVPAANAE0PGUPBDOMJT3VMFLTXA0EFT1B2' # your Foursquare ID
CLIENT_SECRET = '212YUF3ILKXGFIYENNUHGRRIUXIPQI02FVB54QCVIUT11ADL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T0LRXY51HVVKPRWVPAANAE0PGUPBDOMJT3VMFLTXA0EFT1B2
CLIENT_SECRET:212YUF3ILKXGFIYENNUHGRRIUXIPQI02FVB54QCVIUT11ADL


In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 7000 # define radius

search_query = 'Bank'



# create URL
url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,  
    33.7490, 
    -84.3880,
    VERSION,
    search_query,
    radius, 
    LIMIT)
url1

'https://api.foursquare.com/v2/venues/explore?client_id=T0LRXY51HVVKPRWVPAANAE0PGUPBDOMJT3VMFLTXA0EFT1B2&client_secret=212YUF3ILKXGFIYENNUHGRRIUXIPQI02FVB54QCVIUT11ADL&ll=33.749,-84.388&v=20180605&query=Bank&radius=7000&limit=300'

In [18]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5ef4c055882fc7644ea64ee8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Atlanta',
  'headerFullLocation': 'Atlanta',
  'headerLocationGranularity': 'city',
  'query': 'bank',
  'totalResults': 144,
  'suggestedBounds': {'ne': {'lat': 33.81200006300006,
    'lng': -84.31237282080842},
   'sw': {'lat': 33.68599993699994, 'lng': -84.46362717919159}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b1d11d3f964a520950b24e3',
       'name': 'SunTrust Plaza',
       'location': {'address': '303 Peachtree St NE',
        'lat': 33.76238950622084,
        'lng': -84.38596229670338,
        'labeledLatLngs': [{'label': 'display',
          'lat': 33.

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results1['response']['groups'][0]['items']
    
bank_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
bank_venues =bank_venues.loc[:, filtered_columns]

# filter the category for each row
bank_venues['venue.categories'] = bank_venues.apply(get_category_type, axis=1)

# clean columns
bank_venues.columns = [col.split(".")[-1] for col in bank_venues.columns]

bank_venues = bank_venues[~bank_venues.name.str.contains('ATM')]

bank_venues



,name,categories,lat,lng
0,SunTrust Plaza,Bank,33.762390,-84.385962
1,Bank of America,Bank,33.760172,-84.387489
2,Suntrust,Bank,33.754646,-84.389984
4,BB&T,Bank,33.760032,-84.388038
5,Wells Fargo,Bank,33.757481,-84.394650
6,Wells Fargo,Bank,33.738222,-84.417933
7,SunTrust,Bank,33.760945,-84.387535
8,Bank of America,Bank,33.770895,-84.386109
9,SunTrust Bank,Bank,33.755113,-84.400819
10,Wells Fargo,Bank,33.758551,-84.347559


In [21]:
for lat, lng, name in zip(bank_venues['lat'], bank_venues['lng'], bank_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        parse_html=False).add_to(map_atlanta)


In [22]:
map_atlanta

In [23]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 7000 # define radius

search_query = 'Law'



# create URL
url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,  
    33.7490, 
    -84.3880,
    VERSION,
    search_query,
    radius, 
    LIMIT)
url2

results2 = requests.get(url2).json()
results2

venues = results2['response']['groups'][0]['items']
    
law_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
law_venues =law_venues.loc[:, filtered_columns]

# filter the category for each row
law_venues['venue.categories'] = law_venues.apply(get_category_type, axis=1)

# clean columns
law_venues.columns = [col.split(".")[-1] for col in law_venues.columns]

law_venues

,name,categories,lat,lng
0,The Ford Law Firm P.C.,Law School,33.755520,-84.390312
1,Georgia State University Law Library,Law School,33.756638,-84.385813
2,Atlanta's John Marshall Law School,Law School,33.793224,-84.388241
3,Atlanta's John Marshall Law School's PB Adcock...,Law School,33.793624,-84.388201
4,Atlanta's John Marshall Law School: T. C. Sino...,Law School,33.793284,-84.388564
5,The Law Office of Donald P Edwards,Lawyer,33.750887,-84.393410
6,Scheller College Of Business,College Academic Building,33.776262,-84.387893
7,Hornsby Law Group,Lawyer,33.786972,-84.388245
8,N. Sandy Epstein Attorney At Law,Lawyer,33.759534,-84.388448
9,Urban Life Law Library,College Library,33.751327,-84.385887


In [24]:
for lat, lng, name in zip(law_venues['lat'], law_venues['lng'], law_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='black',
        fill_opacity=1,
        parse_html=False).add_to(map_atlanta)

map_atlanta

In [25]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 7000 # define radius

search_query = 'College'



# create URL
url2 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,  
    33.7490, 
    -84.3880,
    VERSION,
    search_query,
    radius, 
    LIMIT)
url2

results2 = requests.get(url2).json()
results2

venues = results2['response']['groups'][0]['items']
    
college_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
college_venues =college_venues.loc[:, filtered_columns]

# filter the category for each row
college_venues['venue.categories'] = college_venues.apply(get_category_type, axis=1)

# clean columns
college_venues.columns = [col.split(".")[-1] for col in college_venues.columns]

college_venues

for lat, lng, name in zip(college_venues['lat'], college_venues['lng'], college_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='black',
        fill_opacity=0.2,
        parse_html=False).add_to(map_atlanta)

map_atlanta

In [26]:
map_atlanta

In [27]:
office_venues = pd.concat([bank_venues, law_venues, college_venues])

In [28]:
office_venues

,name,categories,lat,lng
0,SunTrust Plaza,Bank,33.762390,-84.385962
1,Bank of America,Bank,33.760172,-84.387489
2,Suntrust,Bank,33.754646,-84.389984
4,BB&T,Bank,33.760032,-84.388038
5,Wells Fargo,Bank,33.757481,-84.394650
6,Wells Fargo,Bank,33.738222,-84.417933
7,SunTrust,Bank,33.760945,-84.387535
8,Bank of America,Bank,33.770895,-84.386109
9,SunTrust Bank,Bank,33.755113,-84.400819
10,Wells Fargo,Bank,33.758551,-84.347559


In [29]:
kclusters = 3

areas_grouped_clustering = office_venues.drop('name', 1)
areas_grouped_clustering = areas_grouped_clustering.drop('categories',1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(areas_grouped_clustering)

kmeans.labels_[:]

array([1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 0, 0, 1, 2, 1, 1, 1, 2, 1, 1, 0,
       0, 1, 0, 1, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 2, 1, 1, 1, 2, 0, 0, 0,
       0, 0, 2, 2, 0, 0, 2, 0, 2, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 2, 0,
       0, 0, 0, 1, 0, 1, 1, 2, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 0, 0, 2,
       2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2, 2, 2,
       2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [34]:
centerval = kmeans.cluster_centers_.tolist()

In [35]:
centerval

[[33.78136069993684, -84.39055773830357],
 [33.75464633394136, -84.38157942213931],
 [33.7395597700488, -84.40857624674454]]

In [36]:
potentialloc = pd.DataFrame(centerval)

In [37]:
potentialloc

,0,1
0,33.781361,-84.390558
1,33.754646,-84.381579
2,33.739560,-84.408576


In [38]:
potentialloc.columns = ['lat','lng']

In [39]:
potentialloc

,lat,lng
0,33.781361,-84.390558
1,33.754646,-84.381579
2,33.739560,-84.408576


In [40]:
for lat, lng in zip(potentialloc['lat'], potentialloc['lng']):
    label = 'Potential Location'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1,
        parse_html=False).add_to(map_atlanta)


In [42]:
map_atlanta